This script is meant to process only ONE MONTH'S taxes in Distribution .txt files from the Tax Commission for FISCAL YEAR 2010 ONWARD.

pygsheets can be installed with `pip install pygsheets`

In [3]:
import pandas as pd
import numpy as np
from datetime import date
import pygsheets

This cell immediately below is the only one that requires changes to run for a new month.

In [26]:
#Read in Distribution to Finance .txt files from Tax Commission
tax_distribution_file = './DistributionFiles/2018-07- DISTRIBUTION FILE TO FINANCE.txt'
df = pd.read_fwf(tax_distribution_file, header=None, names=['Date', 'Tax', 'Location', 'Distribution'])
    # dtype={'Date': str, 'Tax': str, 'Location':str, 'Distribution': str}

In [8]:
df.head()

,Date,Tax,Location,Distribution
0,201807SEM,ES,01000Beaver County,656221
1,201807SEM,ES,03038Logan,5581794
2,201807SEM,ES,06000Davis County,7881883
3,201807SEM,ES,06004Bountiful,4393324
4,201807SEM,ES,06008Clearfield,2603077


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 4 columns):
Date            993 non-null object
Tax             993 non-null object
Location        993 non-null object
Distribution    993 non-null int64
dtypes: int64(1), object(3)
memory usage: 31.1+ KB


Translate obscure two letter tax codes to description
Tax code info is stored in a [spreadsheet on Drive](https://docs.google.com/spreadsheets/d/1QA13HliH3YCPgPrmeDLbmw57KCjZbwreSsXGquqmsyU/edit?usp=sharing)

To authorize with the google sheets api you will need to get [oauth client credentials](http://pygsheets.readthedocs.io/en/latest/authorizing.html#oauth-credentials)

In [10]:
import pygsheets
# An oauth client_secret.json is needed in the same directory as this file.
gc = pygsheets.authorize()
# Open  a Google spreadsheet by drive ID
spreadsheet = gc.open_by_key('1QA13HliH3YCPgPrmeDLbmw57KCjZbwreSsXGquqmsyU')
sheet1 = spreadsheet.worksheet_by_title("Sheet1")
# Get all values of sheet as 2d list of cells

# You can get a pandas dataframe if it is easier
sheet_dataframe = sheet1.get_as_df(start=(1, 2), end=(48, 3))

# This will create a dictionary with codes as the key
cell_matrix = sheet1.get_values(start=(2, 2), end=(48, 3), returnas='matrix')
code_lookup = {code: name for (code, name) in cell_matrix}
for code in code_lookup:
    df.replace(code, code_lookup[code], inplace=True)


In [11]:
#Separate Location Code from Location; remove 'SEM' from Date
df['LocationCode'] = df['Location'].str[:5].apply(np.int64) 
df['Location'] = df['Location'].str[5:]
df['Date'] = df['Date'].str[:-3]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 5 columns):
Date            993 non-null object
Tax             993 non-null object
Location        993 non-null object
Distribution    993 non-null int64
LocationCode    993 non-null int64
dtypes: int64(2), object(3)
memory usage: 38.9+ KB


In [12]:
print(df)

       Date                       Tax               Location  Distribution  \
0    201807  E911 Emergency Services           Beaver County        656221   
1    201807  E911 Emergency Services                   Logan       5581794   
2    201807  E911 Emergency Services            Davis County       7881883   
3    201807  E911 Emergency Services               Bountiful       4393324   
4    201807  E911 Emergency Services              Clearfield       2603077   
5    201807  E911 Emergency Services                  Layton       4206227   
6    201807  E911 Emergency Services            Emery County        831114   
7    201807  E911 Emergency Services         Garfield County        445578   
8    201807  E911 Emergency Services            Grand County       1334939   
9    201807  E911 Emergency Services             Juab County       1010879   
10   201807  E911 Emergency Services             Kane County        721723   
11   201807  E911 Emergency Services          Millard County    

In [13]:
#Insert decimal points into Distribution amounts; rename column
df['Distribution'] = df['Distribution'].astype(str)
df.info()
df['Distribution'] = df['Distribution'].str[:-2] + "." + df['Distribution'].str[-2:]
df['Distribution'] = df['Distribution'].astype(np.float64)
df.rename(columns = {'Distribution': 'Dollars_Distributed'}, inplace=True)
df.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 5 columns):
Date            993 non-null object
Tax             993 non-null object
Location        993 non-null object
Distribution    993 non-null object
LocationCode    993 non-null int64
dtypes: int64(1), object(4)
memory usage: 38.9+ KB


,Date,Tax,Location,Dollars_Distributed,LocationCode
0,201807,E911 Emergency Services,Beaver County,6562.21,1000
1,201807,E911 Emergency Services,Logan,55817.94,3038
2,201807,E911 Emergency Services,Davis County,78818.83,6000
3,201807,E911 Emergency Services,Bountiful,43933.24,6004
4,201807,E911 Emergency Services,Clearfield,26030.77,6008
5,201807,E911 Emergency Services,Layton,42062.27,6030
6,201807,E911 Emergency Services,Emery County,8311.14,8000
7,201807,E911 Emergency Services,Garfield County,4455.78,9000
8,201807,E911 Emergency Services,Grand County,13349.39,10000
9,201807,E911 Emergency Services,Juab County,10108.79,12000


In [14]:
#Grab a single date from Date 
year_month = df.iloc[0]['Date']
print(type(year_month))
print(year_month)

<class 'str'>
201807


In [15]:
#Extract year and month individually
year = int(year_month[:-2])
print(type(year))
print(year)
month = int(year_month[-2:])
print(type(month))
print(month)

<class 'int'>
2018
<class 'int'>
7


In [16]:
#Convert the Date to Calendar from Fiscal year
    #FY 2010 month 1 = July 2009
df.rename(columns = {'Date': 'Date_CY'}, inplace=True)

if month == 1:
    month = 7
    year = year - 1
elif month == 2:
    month = 8
    year = year - 1
elif month == 3:
    month = 9
    year = year - 1
elif month == 4:
    month = 10
    year = year - 1
elif month == 5:
    month = 11
    year = year - 1
elif month == 6:
    month = 12
    year = year - 1
    
elif month == 7:
    month = 1
elif month == 8:
    month = 2
elif month == 9:
    month = 3
elif month == 10:
    month = 4
elif month == 11:
    month = 5
elif month == 12:
    month = 6

In [17]:
#Reformat the date
when = date(year, month, 1)
print(when)

2018-01-01


In [18]:
#Apply the formatted date object to entire Date column
df['Date_CY'] = when

In [19]:
df.head(10)

,Date_CY,Tax,Location,Dollars_Distributed,LocationCode
0,2018-01-01,E911 Emergency Services,Beaver County,6562.21,1000
1,2018-01-01,E911 Emergency Services,Logan,55817.94,3038
2,2018-01-01,E911 Emergency Services,Davis County,78818.83,6000
3,2018-01-01,E911 Emergency Services,Bountiful,43933.24,6004
4,2018-01-01,E911 Emergency Services,Clearfield,26030.77,6008
5,2018-01-01,E911 Emergency Services,Layton,42062.27,6030
6,2018-01-01,E911 Emergency Services,Emery County,8311.14,8000
7,2018-01-01,E911 Emergency Services,Garfield County,4455.78,9000
8,2018-01-01,E911 Emergency Services,Grand County,13349.39,10000
9,2018-01-01,E911 Emergency Services,Juab County,10108.79,12000


In [29]:
df.to_csv('./OutputCsv/{}_{}_sales_taxes.csv'.format(year, month))